# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Send HTTP request to the target Website

In [2]:
url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/"

In [3]:
response = requests.get(url)

check the response status code

In [4]:
response

<Response [200]>

### Parse the response content with BeautifulSoap

In [5]:
raw_html = response.text

In [6]:
soup = BeautifulSoup(raw_html, "html.parser")

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [74]:
products = []
page_number = 1
while True:
    url_paged = url + "page/" + str(page_number) + "/"
    response = requests.get(url_paged)
    raw_html = response.text
    soup = BeautifulSoup(raw_html, "html.parser")

    products_on_page = soup.select(".product-wrapper")
    if not products_on_page:
        break;

    for product in products_on_page:
        title = product.select_one(".wd-entities-title").text.strip()
        prices = product.find_all("span", class_="woocommerce-Price-amount")
        if len(prices) == 1:
            regular_price = prices[0].text.strip()
            discount_price = None
        elif len(prices) == 2:
            regular_price = prices[0].text.strip()
            discount_price = prices[1].text.strip()
        else:
            regular_price = None
            discount_price = None
            
        url_to_product_page = product.select_one('.product-image-link').get("href")

        add_to_cart_url = product.select_one('.wd-add-btn a').get("href")
        add_to_cart_url = url + add_to_cart_url

        product_dict = {
            "ProductTitle" : title,
            "RegularPrice" : regular_price,
            "DiscountPrice" : discount_price,
            "URLToProductPage" : url_to_product_page,
            "AddToCartURL" : add_to_cart_url
        }

        products.append(product_dict)

    page_number += 1

In [76]:
len(products)

320

Helper:

In [78]:
test = soup.select(".product-wrapper")
urll = url + "page/" + str(1) + "/"
for t in test:
    naslov = t.select_one(".wd-entities-title").text.strip()
    pr = t.find_all("span", class_="woocommerce-Price-amount")
    if len(pr) ==  1:
        reg = pr[0].text.strip()
        dsc = None
    elif len(pr) == 2:
        reg = pr[0].text.strip()
        dsc = pr[1].text.strip()
    link = t.select_one('.product-image-link').get("href")
    cart = t.select_one('.wd-add-btn a').get("href")
    cart = url + cart
    print(cart)


Repeat the extraction process for each page of products

### Create a pandas DataFrame with the scraped products

In [80]:
df = pd.DataFrame(products)

In [81]:
df

,ProductTitle,RegularPrice,DiscountPrice,URLToProductPage,AddToCartURL
0,Acer A315-23-A7KD,17.590 ден,None,https://clevershop.mk/product/acer-a315-23-a7kd/,https://clevershop.mk/product-category/mobilni...
1,Acer A315-23-R5P2,27.490 ден,None,https://clevershop.mk/product/acer-a315-23-r5p2/,https://clevershop.mk/product-category/mobilni...
2,ACER Aspire 1 A115-22,18.999 ден,15.999 ден,https://clevershop.mk/product/acer-aspire-1-nx...,https://clevershop.mk/product-category/mobilni...
3,Acer Aspire 3 A315-23-R26A,29.990 ден,None,https://clevershop.mk/product/acer-aspire-3-a3...,https://clevershop.mk/product-category/mobilni...
4,Acer Aspire 3 A315-58-33WK,24.490 ден,None,https://clevershop.mk/product/21498/,https://clevershop.mk/product-category/mobilni...
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12.890 ден,None,https://clevershop.mk/product/monitor-27-phili...,https://clevershop.mk/product-category/mobilni...
316,Philips 24″ 243V7QDSB,8.390 ден,None,https://clevershop.mk/product/philips-24%e2%80...,https://clevershop.mk/product-category/mobilni...
317,Philips 27″ 278E1A/00 4K UHD IPS,18.990 ден,None,https://clevershop.mk/product/hp-27%e2%80%b3-2...,https://clevershop.mk/product-category/mobilni...
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26.990 ден,None,https://clevershop.mk/product/philips-279c9-00...,https://clevershop.mk/product-category/mobilni...


Save the dataframe as `.csv`

In [82]:
df.to_csv('products-clever-shop.csv', index=False)